In [59]:
## Tools creation
from langchain_community.tools import ArxivQueryRun,WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper,ArxivAPIWrapper


In [60]:
api_wrapper_wiki=WikipediaAPIWrapper(top_k_results=2,doc_content_chars_max=200)
wiki=WikipediaAPIWrapper(api_wrapper=api_wrapper_wiki)
print(wiki.run("transformers in NLP"))

Page: Generative pre-trained transformer
Summary: A generative pre-trained transformer (GPT) is a type of large language model (LLM) that is widely used in generative AI chatbots. GPTs are based on a deep learning architecture called the transformer. They are pre-trained on large datasets of unlabeled content, and able to generate novel content.
OpenAI was the first to apply generative pre-training to the transformer architecture, introducing the GPT-1 model in 2018. The company has since released many bigger GPT models. The popular chatbot ChatGPT, released in late 2022 (using GPT-3.5), was followed by many competitor chatbots using their own generative pre-trained transformers to generate text, such as Gemini, DeepSeek or Claude.
GPTs are primarily used to generate text, but can be trained to generate other kinds of data. For example, GPT-4o can process and generate text, images and audio. To improve performance on complex tasks, some GPTs, such as OpenAI o3, allocate more computatio

In [61]:
api_wrapper_wiki=ArxivAPIWrapper(top_k_results=2,doc_content_chars_max=200)
arxiv=ArxivAPIWrapper(api_wrapper=api_wrapper_wiki)



In [62]:
tools=[wiki,arxiv]

In [63]:
## custom tools 
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS


In [64]:
import  os 
from dotenv import load_dotenv
load_dotenv()

os.environ["HUGGINGFACEHUB_API_TOKEN"]=os.getenv("HUGGINGFACEHUB_API_TOKEN")
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [65]:
loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()

# 2. Split documents
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)
splits = text_splitter.split_documents(docs)

# 3. Create vector store
vectordb = FAISS.from_documents(
    documents=splits,
    embedding=embeddings
)

# 4. Create retriever
retriever = vectordb.as_retriever()
retriever


VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7f75d8e8d750>, search_kwargs={})

In [66]:
from langchain.tools import tool

@tool
def wiki_search(query: str):
    """Search Wikipedia"""
    return wiki.run(query)

@tool
def langsmith_search(query: str):
    """Search any information about LangSmith"""
    return retriever(query)

@tool
def arxiv_search(query: str):
    """Search Arxiv papers"""
    return arxiv.run(query)

tools = [wiki_search, arxiv_search, langsmith_search]



In [67]:
groq_api_key=os.getenv("GROQ_API_KEY")


In [68]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model_name="llama-3.1-8b-instant",
    api_key=groq_api_key
)



In [69]:

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful AI assistant. "
            "Use the provided tools and context to answer accurately. "
            "If you do not know the answer, say you do not know."
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)




In [70]:
loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()

# 2. Split documents
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)
splits = text_splitter.split_documents(docs)

# 3. Create vector store
vectordb = FAISS.from_documents(
    documents=splits,
    embedding=embeddings
)

# 4. Create retriever
retriever = vectordb.as_retriever()
retriever


VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7f77262c4ad0>, search_kwargs={})

In [71]:
loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()

# 2. Split documents
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)
splits = text_splitter.split_documents(docs)

# 3. Create vector store
vectordb = FAISS.from_documents(
    documents=splits,
    embedding=embeddings
)

# 4. Create retriever
retriever = vectordb.as_retriever()
retriever


VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7f772620bc50>, search_kwargs={})

In [72]:
from langchain.agents import create_agent

model = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0.1,
    max_tokens=1000,
    timeout=30
    # ... (other params)
)
agent = create_agent(model, tools=tools)


In [ ]:
agent.invoke({"messages": [{"type": "human", "content": "what is attention"}]})